In [12]:
from openmeteopy import OpenMeteo
from openmeteopy.options import EcmwfOptions
from openmeteopy.hourly import HourlyEcmwf
from openmeteopy.utils.constants import *
import json
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
# Latitude, Longitude 
latitude =  40
longitude = -88

# Set a class to specify hourly forecasts for the ECMWF provider
hourly = HourlyEcmwf()

# Set the variables that you want to retrieve adding it to the hourly class
hourly = hourly.windspeed_10m()
hourly = hourly.temperature_300hpa()
hourly = hourly.cloudcover()

# Set options to get specific forecasts days and coordinates
options = EcmwfOptions(latitude,
                       longitude,
                       forecast_days=3)

# Set the OpenMeteo client to fetch data
client = OpenMeteo(options, hourly=hourly)

In [7]:
res_json = client.save_json('hourly_weather.json')
print(res_json)

None


In [37]:
def get_and_format_weather_text(latitude, longitude):
    """
    Fetches daily weather data from Open-Meteo API and formats it into a text string.

    Args:
        latitude: Latitude of the location.
        longitude: Longitude of the location.

    Returns:
        A text string containing the formatted weather forecast.
    """
    try:
        # Setup the Open-Meteo API client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
        retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
        openmeteo = openmeteo_requests.Client(session=retry_session)

        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": ["temperature_2m_max", "temperature_2m_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "wind_speed_10m_max"],
            "timezone": "America/Chicago",
            "forecast_days": 10
        }
        responses = openmeteo.weather_api(url, params=params)

        response = responses[0]

        text_output = f"Weather Forecast for\n"
        text_output += f"Timezone: {response.Timezone()} {response.TimezoneAbbreviation()}\n"
        text_output += f"Unit: Temperature - Celsius; daylight or sunshine duration - seconds; precipitation - mm; Wind Speed - m/s\n"

        daily = response.Daily()
        daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
        daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
        daily_sunrise = daily.Variables(2).ValuesAsNumpy()
        daily_sunset = daily.Variables(3).ValuesAsNumpy()
        daily_daylight_duration = daily.Variables(4).ValuesAsNumpy()
        daily_sunshine_duration = daily.Variables(5).ValuesAsNumpy()
        daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
        daily_wind_speed_10m_max = daily.Variables(7).ValuesAsNumpy()

        daily_data = {"date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        )}

        daily_data["temperature_2m_max"] = daily_temperature_2m_max
        daily_data["temperature_2m_min"] = daily_temperature_2m_min
        daily_data["sunrise"] = daily_sunrise
        daily_data["sunset"] = daily_sunset
        daily_data["daylight_duration"] = daily_daylight_duration
        daily_data["sunshine_duration"] = daily_sunshine_duration
        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

        df = pd.DataFrame(daily_data)
        for index, row in df.iterrows():
            date = row['date'].strftime('%Y-%m-%d')
            text_output += f"Date: {date}\n"
            text_output += f"  Max Temperature: {row['temperature_2m_max']}\n"
            text_output += f"  Min Temperature: {row['temperature_2m_min']}\n"
            text_output += f"  Daylight Duration: {row['daylight_duration']}\n"
            text_output += f"  Sunshine Duration: {row['sunshine_duration']}\n"
            text_output += f"  Precipitation: {row['precipitation_sum']}\n"
            text_output += f"  Max Wind Speed: {row['wind_speed_10m_max']}\n"

        return text_output

    except Exception as e:
        return f"An error occurred: {e}"

In [38]:
latitude = 40
longitude = -88
weather_text = get_and_format_weather_text(latitude, longitude)
weather_text

"Weather Forecast for\nTimezone: b'America/Chicago' b'GMT-6'\nUnit: Temperature - Celsius; daylight or sunshine duration - seconds; precipitation - mm; Wind Speed - m/s\nDate: 2025-02-28\n  Max Temperature: 19.23699951171875\n  Min Temperature: -0.06300000101327896\n  Daylight Duration: 40618.13671875\n  Sunshine Duration: 36629.3515625\n  Precipitation: 0.0\n  Max Wind Speed: 45.62351608276367\nDate: 2025-03-01\n  Max Temperature: 2.687000036239624\n  Min Temperature: -5.7129998207092285\n  Daylight Duration: 40775.37109375\n  Sunshine Duration: 18000.0\n  Precipitation: 0.0\n  Max Wind Speed: 30.752090454101562\nDate: 2025-03-02\n  Max Temperature: 4.599999904632568\n  Min Temperature: -9.012999534606934\n  Daylight Duration: 40932.6640625\n  Sunshine Duration: 37303.75390625\n  Precipitation: 0.0\n  Max Wind Speed: 15.480000495910645\nDate: 2025-03-03\n  Max Temperature: 4.849999904632568\n  Min Temperature: -0.5\n  Daylight Duration: 41089.890625\n  Sunshine Duration: 5358.73388671

In [39]:
file = open('weather_updating_context.txt', 'w') 
file.write(weather_text) 
file.close()